In [17]:
import os
import json
import pandas as pd
import re
import wordninja
import pdfplumber
from datetime import datetime, timedelta

os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.tools import Tool
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

In [18]:
with open('config.json') as f:
    config = json.load(f)

OPENAI_API_KEY = config['OPENAI_API_KEY']
PDF_DIR = config.get('DATA_DIR')
PROCESSED_DIR = config.get('PROCESSED_DIR')
VECTOR_STORE_PATH = config.get('VECTORS_DIR')
TRANSACTIONS_CSV = config.get('TRANSACTIONS_CSV')

os.makedirs(PROCESSED_DIR, exist_ok=True)
os.makedirs(VECTOR_STORE_PATH, exist_ok=True)

In [19]:
# ==========================================
# Part 1: PDF Extraction
# ==========================================

DEPOSIT_TRIGS = ['Deposit', 'MB-Transferfrom']

def split_concatenated_text(text):
    return " ".join(wordninja.split(text))

def extract_year(lines):
    for line in lines:
        match = re.search(r'\b(20\d{2})\b', line)
        if match:
            return match.group(1)
    return None

def extract_transactions_from_page(lines, year):
    transactions = []
    months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

    for line in lines:
        line = line.strip()
        if any(line.startswith(month) for month in months):
            parts = line.split()
            if not parts[-1].isdigit() and parts[-1].isalpha():
                del parts [-1]

            date = parts[0]
            if year:
                date = f"{date}{year}"
                try:
                    date = pd.to_datetime(date, format='%b%d%Y').strftime('%b %d, %Y')
                except ValueError:
                    continue
            balance = parts[-1]
            if any(tag in parts for tag in DEPOSIT_TRIGS):
                deposit = parts[-2]
                withdrawal = None
            else:
                deposit = None
                withdrawal = parts[-2]

            description = ' '.join([part for part in parts[1:-2] if not re.match(r'^\d+(\.\d{1,2})?$', part)])
            description = split_concatenated_text(description)
            transactions.append([date, description, withdrawal, deposit, balance])
        elif transactions:
            transactions[-1][1] += ' ' + split_concatenated_text(line)
    return transactions

def process_single_pdf(file_path):
    transactions = []

    with pdfplumber.open(file_path) as pdf:
        year = None

        for page in pdf.pages:
            text = page.extract_text()
            lines = text.split('\n')

            if not year:
                year = extract_year(lines)

            transactions.extend(extract_transactions_from_page(lines, year))

    columns = ['Date', 'Description', 'Withdrawals ($)', 'Deposits ($)', 'Balance ($)']
    return pd.DataFrame(transactions, columns=columns)

def extract_all_pdfs(pdf_directory):
    all_transactions = pd.DataFrame()

    pdf_files = [f for f in os.listdir(pdf_directory) if f.endswith('.pdf')]

    if not pdf_files:
        raise ValueError(f"No PDF files found in {pdf_directory}")

    print(f"\n Found {len(pdf_files)} PDF files to process...")

    for file_name in pdf_files:
        file_path = os.path.join(pdf_directory, file_name)
        print(f"   Processing: {file_name}")
        try:
            transactions = process_single_pdf(file_path)
            all_transactions = pd.concat([all_transactions, transactions], ignore_index=True)
            print(f"   ✓ Extracted {len(transactions)} transactions")
        except Exception as e:
            print(f"   ✗ Error processing {file_name}: {e}")

    return all_transactions


In [20]:
# extract_all_pdfs(PDF_DIR)

In [21]:
# process_single_pdf(PDF_DIR + '/april-2025.pdf')

In [22]:
# ==========================================
# Part 2: Data Processing & Categorization
# ==========================================

def categorize_transaction(description):
    if pd.isna(description):
        return 'other'

    desc_lower = description.lower()

    categories = {
        'groceries': ['grocery', 'instacart', 'safeway', 'walmart', 'superstore', 'sobeys', 'loblaws', 'whole foods', 'metro'],
        'dining': ['restaurant', 'cafe', 'coffee', 'starbucks', 'tim hortons', 'mcdonalds', 'burger', 'pizza', 'subway', 'wendy'],
        'shopping': ['amazon', 'amzn', 'ebay', 'store', 'retail'],
        'transportation': ['uber', 'lyft', 'gas', 'petro', 'shell', 'esso', 'transit', 'parking', 'taxi'],
        'utilities': ['hydro', 'electric', 'gas bill', 'water', 'internet', 'phone', 'rogers', 'bell', 'telus', 'shaw'],
        'entertainment': ['netflix', 'spotify', 'disney', 'cinema', 'theatre', 'game', 'steam', 'playstation', 'xbox'],
        'health': ['pharmacy', 'medical', 'dental', 'doctor', 'hospital', 'clinic', 'shoppers drug'],
        'fitness': ['gym', 'fitness', 'yoga', 'crossfit', 'goodlife', 'planet fitness', 'anytime fitness', 'membership', 'workout'],
        'office': ['staples', 'office', 'supplies'],
        'transfer': ['transfer', 'e-transfer', 'interac'],
    }

    for category, keywords in categories.items():
        if any(keyword in desc_lower for keyword in keywords):
            return category
    return 'other'

def process_transactions(df):
    df['Date'] = pd.to_datetime(df['Date'], format='%b %d, %Y', errors='coerce')

    df['Withdrawals ($)'] = pd.to_numeric(df['Withdrawals ($)'], errors='coerce').fillna(0)
    df['Deposits ($)'] = pd.to_numeric(df['Deposits ($)'], errors='coerce').fillna(0)
    df['Balance ($)'] = pd.to_numeric(df['Balance ($)'], errors='coerce').fillna(0)

    df['Amount'] = df['Withdrawals ($)'] + df['Deposits ($)']
    df['Type'] = df.apply(lambda x: 'Deposit' if x['Deposits ($)'] > 0 else 'Withdrawal', axis=1)
    df['Month'] = df['Date'].dt.to_period('M')
    df['Week'] = df['Date'].dt.to_period('W')
    df['Year'] = df['Date'].dt.year
    df['DayOfWeek'] = df['Date'].dt.day_name()

    df['Category'] = df['Description'].apply(categorize_transaction)

    df = df.dropna(subset=['Date'])
    return df.sort_values('Date', ascending=False).reset_index(drop=True)

In [23]:
# ==========================================
# Part 3: Vector Store Creation
# ==========================================

def create_semantic_documents(transactions_df):
    documents = []

    for idx, row in transactions_df.iterrows():
        amount = row['Withdrawals ($)'] if row['Withdrawals ($)'] > 0 else row['Deposits ($)']
        trans_type = 'spent' if row['Withdrawals ($)'] > 0 else 'received'

        # Create natural language description
        content = f"""Transaction on {row['Date'].strftime('%B %d, %Y')} ({row['DayOfWeek']}):
                        You {trans_type} ${amount:.2f} at {row['Description']}.
                        Category: {row['Category']}
                        Transaction Type: {row['Type']}
                        Balance after transaction: ${row['Balance ($)']:.2f}
                    """

        documents.append(Document(
            page_content=content,
            metadata={
                'date': row['Date'].strftime('%Y-%m-%d'),
                'description': row['Description'],
                'amount': float(amount),
                'category': row['Category'],
                'type': row['Type'],
                'balance': float(row['Balance ($)']),
                'month': str(row['Month']),
                'week': str(row['Week']),
                'year': int(row['Year']),
                'day_of_week': row['DayOfWeek']
            }
        ))

def create_vector_store(documents, vector_store_path):
    print(f"\nCreating vector embeddings for {len(documents)} transactions...")
    embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
    vectorstore = FAISS.from_documents(documents=documents, embedding=embeddings)
    vectorstore.save_local(vector_store_path)
    print(f"   ✓ Vector store saved to {vector_store_path}")
    return vectorstore

    return documents

def load_vector_store(vector_store_path):
    embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
    return FAISS.load_local(
        folder_path=vector_store_path,
        embeddings=embeddings,
        allow_dangerous_deserialization=True
    )

In [24]:
# ==========================================
# Part 4: Structured Query Engine
# ==========================================

def query_structured_data(query: str, df: pd.DataFrame) -> str:
    query_lower = query.lower()
    today = datetime.now()

    # Filter by time period
    filtered_df = df.copy()
    time_period = "in your records"

    if 'last week' in query_lower or 'past week' in query_lower:
        week_ago = today - timedelta(days=7)
        filtered_df = df[df['Date'] >= week_ago]
        time_period = "last week"
    elif 'last month' in query_lower or 'past month' in query_lower:
        month_ago = today - timedelta(days=30)
        filtered_df = df[df['Date'] >= month_ago]
        time_period = "last month"
    elif 'this month' in query_lower or 'current month' in query_lower:
        filtered_df = df[df['Date'].dt.month == today.month]
        time_period = "this month"
    elif 'this year' in query_lower or 'current year' in query_lower:
        filtered_df = df[df['Date'].dt.year == today.year]
        time_period = "this year"
    elif 'last year' in query_lower:
        last_year = today.year - 1
        filtered_df = df[df['Date'].dt.year == last_year]
        time_period = "last year"

    # Filter by category
    category = None
    for cat in ['groceries', 'grocery', 'dining', 'shopping', 'transportation',
                'utilities', 'entertainment', 'health', 'office', 'transfer']:
        if cat in query_lower:
            category = cat if cat != 'grocery' else 'groceries'
            filtered_df = filtered_df[filtered_df['Category'] == category]
            break

    # Filter by transaction type
    if 'withdrawal' in query_lower or 'spent' in query_lower or 'paid' in query_lower:
        filtered_df = filtered_df[filtered_df['Withdrawals ($)'] > 0]
    elif 'deposit' in query_lower or 'received' in query_lower or 'income' in query_lower:
        filtered_df = filtered_df[filtered_df['Deposits ($)'] > 0]

    if filtered_df.empty:
        return f"No transactions found {time_period}" + (f" for {category}" if category else "")

    # Handle different query types
    if any(word in query_lower for word in ['total', 'sum', 'how much', 'amount']):
        total_spent = filtered_df['Withdrawals ($)'].sum()
        total_received = filtered_df['Deposits ($)'].sum()
        count = len(filtered_df)

        result = f"**Summary {time_period}"
        if category:
            result += f" for {category}"
        result += ":**\n\n"

        if 'spent' in query_lower or 'withdrawal' in query_lower or category:
            result += f"Total spent: ${total_spent:,.2f}\n"
            result += f"Number of transactions: {count}\n"
            result += f"Average per transaction: ${total_spent/count:,.2f}\n\n"
        elif 'deposit' in query_lower or 'received' in query_lower:
            result += f"Total received: ${total_received:,.2f}\n"
            result += f"Number of transactions: {count}\n\n"
        else:
            result += f"Total spent: ${total_spent:,.2f}\n"
            result += f"Total received: ${total_received:,.2f}\n"
            result += f"Net: ${total_received - total_spent:,.2f}\n"
            result += f"Number of transactions: {count}\n\n"

        # Show top transactions
        result += "**Top transactions:**\n"
        top_transactions = filtered_df.nlargest(5, 'Amount')[['Date', 'Description', 'Amount']]
        for _, row in top_transactions.iterrows():
            result += f"• {row['Date'].strftime('%b %d, %Y')}: ${row['Amount']:,.2f} at {row['Description']}\n"

        return result

    elif 'average' in query_lower or 'mean' in query_lower:
        avg_spent = filtered_df['Withdrawals ($)'].mean()
        result = f"Average spent {time_period}"
        if category:
            result += f" on {category}"
        result += f": ${avg_spent:,.2f}"
        return result

    elif 'largest' in query_lower or 'biggest' in query_lower or 'most expensive' in query_lower or 'highest' in query_lower:
        largest = filtered_df.nlargest(5, 'Amount')
        result = f"**Largest transactions {time_period}:**\n\n"
        for _, row in largest.iterrows():
            result += f"• {row['Date'].strftime('%b %d, %Y')}: ${row['Amount']:,.2f} at {row['Description']}\n"
        return result

    elif 'smallest' in query_lower or 'lowest' in query_lower:
        smallest = filtered_df[filtered_df['Amount'] > 0].nsmallest(5, 'Amount')
        result = f"**Smallest transactions {time_period}:**\n\n"
        for _, row in smallest.iterrows():
            result += f"• {row['Date'].strftime('%b %d, %Y')}: ${row['Amount']:,.2f} at {row['Description']}\n"
        return result

    elif 'count' in query_lower or 'how many' in query_lower or 'number' in query_lower:
        count = len(filtered_df)
        result = f"You made {count} transactions {time_period}"
        if category:
            result += f" in the {category} category"
        return result

    elif 'breakdown' in query_lower or 'category' in query_lower or 'categories' in query_lower:
        category_breakdown = filtered_df.groupby('Category')['Withdrawals ($)'].sum().sort_values(ascending=False)
        result = f"**Category breakdown {time_period}:**\n\n"
        for cat, amount in category_breakdown.items():
            percentage = (amount / filtered_df['Withdrawals ($)'].sum()) * 100
            result += f"• {cat.title()}: ${amount:,.2f} ({percentage:.1f}%)\n"
        return result

    result = f"**Recent transactions {time_period}"
    if category:
        result += f" - {category}"
    result += ":**\n\n"

    recent = filtered_df.head(10)[['Date', 'Description', 'Amount', 'Balance ($)']]
    for _, row in recent.iterrows():
        result += f"• {row['Date'].strftime('%b %d, %Y')}: ${row['Amount']:,.2f} at {row['Description']} (Balance: ${row['Balance ($)']:,.2f})\n"

    return result

In [25]:
# ==========================================
# Part 5: Complete RAG System Setup
# ==========================================

class BankStatementRAG:

    def __init__(self, force_refresh=True):
        self.transactions_df = None
        self.vectorstore = None
        self.agent = None


        needs_extraction = force_refresh or not os.path.exists(TRANSACTIONS_CSV)
        needs_vectors = force_refresh or not os.path.exists(os.path.join(VECTOR_STORE_PATH, 'index.faiss'))

        if needs_extraction:
            print("Starting fresh extraction from PDFs...")
            self._extract_and_process()
        else:
            print("Loading existing transactions...")
            self.transactions_df = pd.read_csv(TRANSACTIONS_CSV)
            self.transactions_df = process_transactions(self.transactions_df)
            print(f"   ✓ Loaded {len(self.transactions_df)} transactions")

        if needs_vectors:
            print("Building vector store...")
            self._build_vector_store()
        else:
            print("Loading existing vector store...")
            self.vectorstore = load_vector_store(VECTOR_STORE_PATH)
            print("   ✓ Vector store loaded")

        self._setup_agent()
        print("\nRAG system ready!\n")

    def _extract_and_process(self):
        raw_df = extract_all_pdfs(PDF_DIR)
        print(f"\n✓ Total transactions extracted: {len(raw_df)}")

        self.transactions_df = process_transactions(raw_df)

        self.transactions_df.to_csv(TRANSACTIONS_CSV, index=False)
        print(f"✓ Processed transactions saved to {TRANSACTIONS_CSV}")

    def _build_vector_store(self):
        documents = create_semantic_documents(self.transactions_df)
        self.vectorstore = create_vector_store(documents, VECTOR_STORE_PATH)

    def _setup_agent(self):

        def structured_query_tool(query: str) -> str:
            return query_structured_data(query, self.transactions_df)

        def semantic_search_tool(query: str) -> str:
            docs = self.vectorstore.similarity_search(query, k=5)
            result = "🔍 **Relevant transactions:**\n\n"
            for doc in docs:
                result += doc.page_content + "\n\n"
            return result

        tools = [
            Tool(
                name="StructuredQuery",
                func=structured_query_tool,
                description="Use this for queries about totals, sums, averages, counts, breakdowns, or any aggregation. Also use for time-based queries like 'last week', 'this month', or category queries. Examples: 'total spent on groceries', 'how much did I spend last month', 'average transaction amount', 'category breakdown'."
            ),
            Tool(
                name="SemanticSearch",
                func=semantic_search_tool,
                description="Use this to find specific transactions by merchant name, description, or when looking for particular types of purchases. Examples: 'transactions at Amazon', 'when did I buy coffee', 'show me Uber rides'."
            )
        ]

        llm = ChatOpenAI(
            temperature=0,
            openai_api_key=OPENAI_API_KEY,
            model="gpt-4o-mini"
        )

        prompt = ChatPromptTemplate.from_messages([
            ("system", """
                You are a helpful financial assistant analyzing bank transactions.

                You have access to two powerful tools:
                1. StructuredQuery: For calculations, aggregations, summaries, and filtering (totals, averages, counts, breakdowns)
                2. SemanticSearch: For finding specific transactions by merchant or description

                Guidelines:
                - Use StructuredQuery for questions about "how much", "total", "sum", "average", "breakdown", time periods, or categories
                - Use SemanticSearch for finding specific merchants or transaction details
                - Always provide clear, formatted answers with dollar amounts and dates
                - Be conversational and helpful
                - Use emojis to make responses engaging
                """
             ),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ])

        agent = create_openai_functions_agent(llm, tools, prompt)
        self.agent = AgentExecutor(
            agent=agent,
            tools=tools,
            verbose=False,
            handle_parsing_errors=True
        )

    def ask(self, question: str) -> str:
        result = self.agent.invoke({"input": question})
        return result['output']

    def get_summary_stats(self):
        total_transactions = len(self.transactions_df)

        if total_transactions == 0:
            return {
                'total_transactions': 0,
                'total_spent': 0,
                'total_received': 0,
                'net': 0,
                'date_range': 'No data',
                'categories': {}
            }

        total_spent = self.transactions_df['Withdrawals ($)'].sum()
        total_received = self.transactions_df['Deposits ($)'].sum()
        date_range = f"{self.transactions_df['Date'].min().strftime('%b %d, %Y')} to {self.transactions_df['Date'].max().strftime('%b %d, %Y')}"
        categories = self.transactions_df['Category'].value_counts().to_dict()

        return {
            'total_transactions': total_transactions,
            'total_spent': total_spent,
            'total_received': total_received,
            'net': total_received - total_spent,
            'date_range': date_range,
            'categories': categories
        }

In [27]:
def main():
    """Main entry point."""
    print("\n" + "="*60)
    print("Bank Statement RAG System")
    print("="*60)

    # Initialize system
    rag = BankStatementRAG(force_refresh=False)

    # Show summary
    stats = rag.get_summary_stats()
    print(f"\nAccount Summary:")
    print(f"   • Total transactions: {stats['total_transactions']}")
    print(f"   • Total spent: ${stats['total_spent']:,.2f}")
    print(f"   • Total received: ${stats['total_received']:,.2f}")
    print(f"   • Net: ${stats['net']:,.2f}")
    print(f"   • Date range: {stats['date_range']}")
    print(f"   • Categories: {len(stats['categories'])}")

    # Interactive mode
    print("\nAsk me anything about your transactions! (type 'quit' to exit)\n")

    while True:
        question = input("You: ").strip()

        if question.lower() in ['quit', 'exit', 'q']:
            print("\nGoodbye!")
            break

        if not question:
            continue

        try:
            answer = rag.ask(question)
            print(f"\nAssistant: {answer}\n")
        except Exception as e:
            print(f"\nError: {e}\n")

if __name__ == "__main__":
    main()


Bank Statement RAG System
Loading existing transactions...
   ✓ Loaded 0 transactions
Loading existing vector store...
   ✓ Vector store loaded

RAG system ready!


Account Summary:
   • Total transactions: 0
   • Total spent: $0.00
   • Total received: $0.00
   • Net: $0.00
   • Date range: No data
   • Categories: 0

💬 Ask me anything about your transactions! (type 'quit' to exit)


Assistant: Here’s a summary of your recent transactions with Amazon 🛒:

### Recent Amazon Transactions:
1. **August 26, 2024**
   - **Amount:** $20.15
   - **Details:** AMZN Mktp CA*R41B22KP0

2. **August 27, 2024**
   - **Amount:** $55.99
   - **Details:** AMZN Mktp CA*RK6RI0E02

3. **August 27, 2024**
   - **Amount:** $67.20
   - **Details:** AMZN Mktp CA*R41O39940

4. **August 27, 2024**
   - **Amount:** $6.69
   - **Details:** AMZN Mktp CA*R40RY19Y0

5. **August 27, 2024**
   - **Amount:** $11.14
   - **Details:** AMZN Mktp CA*R42UX4IE0

6. **September 19, 2024**
   - **Amount:** $67.19
   - **Detail

KeyboardInterrupt: Interrupted by user